In [225]:
pip list --format=freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [215]:
from jupyter_dash import JupyterDash
import dash_cytoscape as cyto
from dash import html, dcc, Input, Output
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
import statsmodels.api as sm

## Heatwave Map

In [216]:
df_heatwave = pd.read_csv('https://github.com/hongdoudahongdou/MDA/blob/main/data/heatwave.csv?raw=true')
fig_map_heatwave=px.choropleth(df_heatwave , locations="ISO",color="Disaster Subtype",animation_frame="Year")
fig_map_heatwave

## Total Death Map

In [217]:
df_death=pd.read_csv("https://github.com/hongdoudahongdou/MDA/blob/main/data/df_death.csv?raw=true")
fig_death_map=px.choropleth(df_death, locations="ISO",color="Total Deaths",animation_frame="Year")
fig_death_map

In [218]:
#nb_model
df1=pd.read_csv('https://github.com/hongdoudahongdou/MDA/blob/main/data/train_data.csv?raw=true')
x = df1[['log_GDP(million dollars)','log_Population(e+07)','Health expenditure (% of GDP)','max_temperature','Associated Drought','Associated Wildfire',
        'Appeal or Declaration','duration']]
y = df1[['Total Deaths']]
model = sm.GLM(y,x,family=sm.families.NegativeBinomial())
results = model.fit()
def pred_death(x):
    return round(results.predict(x))

In [219]:
#prepare the data
df_2020=pd.read_csv("https://github.com/hongdoudahongdou/MDA/blob/main/data/df_2020.csv/?raw=true")
df_new = df_2020[df_2020['ISO'].isin(df_heatwave['ISO'])]
df_new = df_new.dropna()
df_new['log_GDP(million dollars)']=df_new['GDP(million dollars)'].apply(np.log)
df_new['log_Population(e+07)']=df_new['Population'].apply(np.log)
df_new.drop(['GDP(million dollars)','Population'],axis=1,inplace=True)
df_new['duration']=5
df_new

,Country,ISO,Health expenditure (% of GDP),log_GDP(million dollars),log_Population(e+07),duration
13,Australia,AUS,10.648995,27.916200,17.060260,5
14,Austria,AUT,11.474260,26.798612,16.003455,5
17,Belgium,BEL,11.064532,26.986664,16.261209,5
20,Bangladesh,BGD,2.628010,26.647522,18.936022,5
21,Bulgaria,BGR,8.520638,24.977521,15.751950,5
24,Bosnia and Herzegovina,BIH,9.841127,23.730236,15.014995,5
28,Bolivia,BOL,7.857059,24.324129,16.295083,5
29,Brazil,BRA,10.312917,28.023063,19.177724,5
35,Canada,CAN,12.939675,28.131796,17.453285,5
37,Switzerland,CHE,11.796350,27.329916,15.971701,5


## Temperature Prediction

In [197]:
df_tem_pre= pd.read_csv('https://github.com/hongdoudahongdou/MDA/blob/main/data/temperature_result.csv?raw=true')
df_tem_pre['Date']=pd.date_range(start='2022-06-30', end='2023-06-30')
df_tem_pre.rename(columns={'tmax': 'True value'}, inplace=True)
fig_tem_pre = go.Figure()
fig_tem_pre.add_trace(go.Scatter(x=df_tem_pre['Date'], y=df_tem_pre['True value'], mode='lines', name='True value'))
fig_tem_pre.add_trace(go.Scatter(x=df_tem_pre['Date'], y=df_tem_pre['Predicted_ARIMA'], mode='lines', name='Predicted_ARIMA'))
fig_tem_pre.add_trace(go.Scatter(x=df_tem_pre['Date'], y=df_tem_pre['Predicted_rf'], mode='lines', name='Predicted_RandomForest'))
fig_tem_pre

In [51]:
df_tem_pre['ARIMA_ERR']=df_tem_pre['Predicted_ARIMA']-df_tem_pre['True value']
df_tem_pre['RF_ERR']=df_tem_pre['Predicted_rf']-df_tem_pre['True value']
fig_pre_err = go.Figure()
fig_pre_err.add_trace(go.Scatter(x=df_tem_pre['Date'], y=df_tem_pre['ARIMA_ERR'], mode='lines', name='ARIMA_ERROR'))
fig_pre_err.add_trace(go.Scatter(x=df_tem_pre['Date'], y=df_tem_pre['RF_ERR'], mode='lines', name='RandomForest_ERROR'))
fig_pre_err

## APP

In [224]:
app=JupyterDash(__name__,suppress_callback_exceptions=True)
dates = pd.date_range(start="2022-07-02", end="2023-06-30")

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#119DFF',
    'color': 'white',
    'padding': '6px'
}

app.layout = html.Div([
    html.H1('MDA Project Heatwave', style={'textAlign': 'center'}),
    html.H3('Yang Zhao R0823619', style={'textAlign': 'center'}),
    html.Div(
        [html.Div([
            html.Img(
                src='https://github.com/hongdoudahongdou/MDA/blob/main/APP/coverpic.jpg?raw=true',style={
        'height': '300px',  
        'width': '400px'   
    })
        ])
        ], style={'textAlign': 'center'}),
    dcc.Tabs(id="tabs-inline", value='tab-1', parent_className='custom-tabs', className='custom-tabs-container',
             children=[
                 dcc.Tab(label='Homepage', value='Homepage', style=tab_style, selected_style=tab_selected_style),
                 dcc.Tab(label='Predict Deaths', value='Predict Deaths', style=tab_style,
                         selected_style=tab_selected_style),
                 dcc.Tab(label='Predict Heatwaves', value='Predict Heatwaves', style=tab_style,
                         selected_style=tab_selected_style)
                      
             ], style=tabs_styles),
    html.Div(id='tabs-content'),

])


@app.callback(Output('tabs-content', 'children'),
              Input('tabs-inline', 'value'))
def render_content(tab):
    if tab == 'Homepage':
        return html.Div([
            html.H1('MDA Project Heatwave', style={'textAlign': 'center'}),
            html.H3('These years, public are paying more and more attention to heat waves as such '
                    'extreme meteorological may could lead to massive deaths . '
                    'In order to have a better knowledge on the damage due to heat waves, '
                    'our group decide to predict the heatwave and deaths caused by heatwaves.'),

            html.Div([
                dcc.Graph(id='World Heatwave',figure=fig_map_heatwave),

                ], style={'width': '60%', 'textAlign': 'center', 'float': 'center', 'display': 'inline',
                           'margin-left': '-300px', 'margin-right': '-300px'}),
            html.H6('Worldwide Heatwave Map', style={'textAlign': 'center'})
        ])
    
    elif tab == 'Predict Deaths':
        return html.Div([
            html.Div([
                html.H1('Predict deaths', style={'textAlign': 'center'}),
                html.H3('World Heatwave Deaths Map',style={'textAlign': 'center'}),
                html.Div([
                    dcc.Graph(id='World Heatwave Deaths', figure=fig_death_map
                                                                        ),

                ], style={'width': '60%', 'textAlign': 'center', 'float': 'center', 'display': 'inline',
                          'margin-left': '-300px', 'margin-right': '-300px'})]),
              ## 
              html.H3('Predict Deaths Map', style={'textAlign': 'center'}),
              html.Div([
                    html.H5('Associate Drought', style={'textAlign': 'center'}),
                    dcc.Dropdown(id='dropdown-drought',options=[{'label': i, 'value': i} for i in [0, 1]],value=0),

                ], style={'width': '33%', 'display': 'inline-block'}),
              html.Div([
                    html.H5('Associate Wildfire', style={'textAlign': 'center'}),
                    dcc.Dropdown(id='dropdown-wildfire',options=[{'label': i, 'value': i} for i in [0, 1]],value=0),

                ], style={'width': '33%', 'display': 'inline-block'}),
              html.Div([
                    html.H5('Appeal or Declare', style={'textAlign': 'center'}),
                    dcc.Dropdown(id='dropdown-appeal',options=[{'label': i, 'value': i} for i in [0, 1]],value=0),

                ], style={'width': '33%', 'display': 'inline-block'}),
           html.Div([
                dcc.Graph(id='Predict deaths map'),

                dcc.Slider(id='slider-temp',min=30,max=45,value=30,step=1
                    

                )], style={'width': '48%', 'textAlign': 'center', 'float': 'center', 'display': 'inline',
                           'margin-left': '-300px', 'margin-right': '-300px'}),
        ])

    
    
    
    
    elif tab == 'Predict Heatwaves':
        return html.Div([
            html.H1('Predict Heatwaves', style={'textAlign': 'center'}),
            html.H3('Belgium defines a heat wave as a period of at least five consecutive days in which the maximum' 
                    'temperature exceeds 25 °C. During this period the maximum temperature must exceed 30 °C for at least three days.'
                    'Based on this defination, we can predict the occurance of heatwaves by predicting temperature. '
                   'We use 2 different model to predict the temperature. One is the ARIMA model, which can give'
                   'a precise prediction of the prediction of tomorrow. The other is Random Forest, which utilize'
                   'the temperature of the last 100 days to predict the temperature of 7 days from now.'),
            html.Div([
                dcc.Graph(id='World Heatwave Deaths', figure=fig_tem_pre),

                ], style={'width': '60%', 'textAlign': 'center', 'float': 'center', 'display': 'inline',
                          'margin-left': '-300px', 'margin-right': '-300px'}),
                
            html.H3('The following chart provides a more intuitive representation of the prediction errors of the two models.'),
            html.Div([
                html.Div([
                dcc.Graph(id='Heatwavetemperaturem',figure=fig_pre_err),
                    ])
                 ], style={'textAlign': 'center'}),
            html.H3('The following is an AI for temperature prediction based on our results.'),
            html.Div([
                dcc.Dropdown(
                id='date-dropdown',
                
                options=[{'label': date.strftime('%Y-%m-%d'), 'value': date.strftime('%Y-%m-%d')} for date in dates],
                value=dates[0].strftime('%Y-%m-%d')
        )
    ], style={'width': '48%', 'display': 'inline-block'}),
        html.Div([
        html.Div(id='output1'),
        html.Div(id='output2'),
        html.Div(id='output3')
    ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
])   
    
    

@app.callback(
    Output('Predict deaths map', 'figure'),
    [
        Input('dropdown-drought', 'value'),
        Input('dropdown-wildfire', 'value'),
        Input('dropdown-appeal', 'value'),
        Input('slider-temp', 'value'),
    ]
)
def update_map(drought, wildfire, appeal, temp):
    df_new['Associated Drought']=drought
    df_new['Associated Wildfire']=wildfire
    df_new['Appeal or Declaration']=appeal
    df_new['max_temperature']=temp
    data = df_new[['log_GDP(million dollars)','log_Population(e+07)','Health expenditure (% of GDP)','max_temperature','Associated Drought','Associated Wildfire',
        'Appeal or Declaration','duration']]
    pred_death(data)
    df_new['Prediction']=pred_death(data)
    fig_death_pre_map=px.choropleth(df_new, locations="ISO",color="Prediction",range_color=(0, 5000))
    return fig_death_pre_map
 


@app.callback(
    [Output('output1', 'children'),
     Output('output2', 'children'),
     Output('output3', 'children')],
    [Input('date-dropdown', 'value')]
)
def update_output(selected_date):
    selected_row = df_tem_pre[df_tem_pre['Date'] == pd.to_datetime(selected_date)].iloc[0]
    ARIMA_value = round(selected_row['Predicted_ARIMA'],2)
    RF_value = round(selected_row['Predicted_rf'],2)
    output1= f"One-day-ahead Prediction for {selected_date} is {ARIMA_value}"
    output2= f"Seven-day-ahead Prediction for {selected_date} is {RF_value}"
    if ARIMA_value >= 25 and RF_value >= 25:
        output3 = "Please be cautious against heat."
          
    else:
        output3 = "The weather is suitable."

    return output1, output2, output3



if __name__ == '__main__':
    app.run_server(debug=True, threaded=True,dev_tools_ui=False,dev_tools_props_check=False)




Dash app running on http://127.0.0.1:8050/
